
# Trends in Music Post 2000s: What Makes Popular Music Popular?

###### Last updated on May 15th, 2022 by Matthew Lynch



## “Pop” music vs “Popular” music?

When people use the term “*pop music*” they could be referring either to popular music (as in music that is popular or trending) or a more specific genre distinct from metal, jazz, rap, indie, and other established genres. The discussion as to what constitutes the genre of pop music is a complicated one as the genre continuously evolves–branching out and [borrowing musical elements from other styles like rock, dance, Latin, and country music](https://en.wikipedia.org/wiki/Pop_music). The genre of pop also typically aligns with what is currently popular at a given time and Wikipedia suggests *“that the term "pop music" may be used to describe a distinct genre, designed to appeal to all.”* Yet when examining the top hits from the last two decades, there are some tracks that probably wouldn’t be considered as emblematic of the “pop” genre, but rather a subgenre or a different style altogether.

The discussion of popular music is also complex as the current notion of *“good”* music relies heavily on subjective opinions influenced by our culture today. The types of songs that are popular in the United States in 2022 differs from the hit songs of 2005 and might not even be the same as what is currently trending in other countires like Spain or South Korea. Because of this I want to make it clear that the goal of this project is **not to define what** ***"good"*** **music is** but merely to examine trends in the most frequently played tracks.

To clarify, for this project rather than analyzing music from a particular genre, I am analyzing **popular music in the United States from January 1st, 2000 to today** (March 16th, 2022) with the intent of determining if certain combinations of musical qualities are more likely to create hit songs than others. While the process of creating music isn't necessarily so straightforward, information about what the most popular song duration, tempo, key signature, or even volume could provide a guideline for people who wish to create their own popular music.


##### *Brief Disclaimer*

At the time of writing this I have very limited experience with music theory. I have taken one course in music theory that covered the fundamentals and I play the guitar as a hobby. As my knowledge of music theory grows, I intend on expanding the scope of this project to analyze the more complex elements of music composition to provide more interesting data on recent trends.


## Billboard Hot 100

To determine which songs are the most popular, I chose to use [Billboard charts](https://www.billboard.com/charts/). I wanted data on the most popular songs within the United States and for how long they have been popular without relying on a specific streaming service. While music streaming services keep track of how often a song is played, managing to collect the total number of times a given track has been played on different services would be an incredibly time consuming task. When searching for an alternative, I found [Billboard Magazine](https://www.billboard.com/) which focuses its brand on constructing charts and reporting on music news and trends across different genres of music. The Billboard Hot 100 acts as a [music industry standard record chart](https://en.wikipedia.org/wiki/Billboard_Hot_100) and provides information taking into considering all of the most common ways to consume music today. The tooltip on the Hot 100 Charts says the following:

> The week’s most popular songs, ranked by audio and video streaming activity on leading digital music services, radio airplay audience impressions based on monitored airplay and sales data, all compiled by Luminate. Audience totals are derived, in part, using certain Nielsen Audio listener estimates.”

They also provide a variety of charts such as the Billboard Global 200, Billboard Global Excluding US, Hot Country Songs, Top Latin Albums, and more.


## Spotify Web Api




## Data Collection

Now with all of the basic information covered, we can now go ahead with obtaining the data.



### Collecting Billboard Top 100 Data

To begin, we first want to scrape the information from the Billboard Hot 100 pages. We're looking to obtain the ranking of each song, the song titles, the artist names, and the number of weeks the song has been in the Hot 100. 

In [278]:
# Import libraries for data collection
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup as bs
import datetime as dt
import re

# Returns a formatted string from a datetime to use when scraping Billboard charts
def format_datetime(datetime):
    return str(datetime.year).zfill(4) + "-" + str(datetime.month).zfill(2) + "-" + str(datetime.day).zfill(2) 

# Returns a formatted string given a string containing artist name(s)
def format_name(name):
    # Fixes the formatting of the ampersand
    amp = re.sub('&amp;', "&", name)
    # Standardizes ways of featuring an artist to "ft."
    ft = re.sub('(Featuring)|(featuring)|(feat\.?)', "ft.", amp)
    return ft

# Removes html tags from a string
def remove_tags(tag, string):
    tag1 ='<' + tag + '.*?>\s*'
    tag2 = '\s*</' + tag + '.*?>'
    return re.sub(tag2, "", re.sub(tag1, "", string))

# Scrapes data from a specified billboard page in a given period of time
def scrape_billboard(start_date, end_date, page):
    info_list = []
    date = start_date
    # Continues to scrape from new pages until the end date is reached
    while date <= end_date:
        # Access the proper url given the date
        billboard_url = "https://www.billboard.com/"  + page + format_datetime(date) + "/"
        soup = bs(rq.get(billboard_url).content)
        # Look for individual song entries
        charts = soup.find_all("div", class_=re.compile('o-chart-results-list-row-container'))
        for entry in charts:
            # Scrape the data from the chart
            rank = remove_tags("span", str(entry.find("span", class_=re.compile('c-label a-font-primary-bold-l'))))
            title = remove_tags("h3", str(entry.find("h3", class_=re.compile('c-title'))))
            artist = remove_tags("span", str(entry.find("span", class_=re.compile('c-label a-no-trucate'))))
            # Properly format the title and artist for ease of use later
            title = format_name(title)
            artist = format_name(artist)
            # Find Last_Week, Peak_Pos, and Wks_on_Chart info
            search = entry.find_all("span", class_=re.compile('(c-label a-font-primary-m lrv-u-padding-tb-050@mobile-max)|(c-label a-font-primary-bold-l a-font-primary-m@mobile-max u-font-weight-normal@mobile-max)'))
            "c-label a-font-primary-bold-l a-font-primary-m@mobile-max u-font-weight-normal@mobile-max"
            # Grab the data on the weeks on the chart
            weeks = remove_tags("span", str(search[2]))
            # Add the data to the info_list
            page_name = "Weeks_in_" + re.sub('charts/|/', "_", page).strip("_")
            data = {'Rank': rank, 'Title': title, 'Artist': artist, 'Week': date, page_name: weeks}
            info_list.append(data)
        # Increment the date by a week (Billboard's charts are on a weekly basis)
        date += dt.timedelta(days = 7)
    # Return a dataframe from the info_list
    return pd.DataFrame(info_list)

# Scrape hot-100 data from 01-01-2000 to today
billboard_data = scrape_billboard(dt.date(2000, 1, 1), dt.date.today(), "charts/hot-100/")
# Track the first and last week a track enters the hot-100 charts instead of each week that the data is in the hot-100
billboard_data.insert(5, "First_Week", billboard_data['Week'], False)
billboard_data.insert(6, "Last_Week", billboard_data['Week'], False)
billboard_data.drop(columns=['Week'], inplace=True)
# Export to a csv to save time in subsequent calls
billboard_data.to_csv("csv/billboard_data.csv")
# Drop the extra row added when converting to csv
billboard_data = pd.read_csv("csv/billboard_data.csv").iloc[:, 1:]
billboard_data.head(10)

Explanation of scraping billboard data:
Why Hot-100 instead of another chart?
Why use First/Last Week markers?
Why format the names?
Why 2000?

### Collecting Spotify Data Based On Billboard Data

In [281]:
aggregation_functions = {'Rank': "min", 'Artist': "first", 'Weeks_in_hot-100': "max", 'First_Week': "min", 'Last_Week': "max"}
spotify_data = billboard_data.groupby(billboard_data['Title']).aggregate(aggregation_functions).reset_index()
spotify_data.insert(0, "Top_Rank", spotify_data['Rank'], False)
spotify_data.drop(columns=['Rank'], inplace=True)
#spotify_data.sort_values(by='Top_Rank').head(10)

In [282]:
# pip install spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import config

authentication = SpotifyClientCredentials(client_id=config.cid, client_secret=config.csecret)
sp = spotipy.Spotify(client_credentials_manager=authentication)

def get_audio_analysis(artist, title):
    q = "{} artist:{}".format(title, artist)
    #print(q)
    result = sp.search(q, type='track', limit=1)['tracks']['items']
    if result == []:
        q = "track:{} artist:{}".format(title, artist)
        result = sp.search(q, type='track', limit=1)['tracks']['items']
        if result == []:
            return [None, None, None, None, None, None, None, None, None, None]
    spotify_id = result[0]['id']
    try:
        analysis = sp.audio_analysis(spotify_id)
    except spotipy.client.SpotifyException:
        analysis = []
    if analysis == []:
        return [None, None, None, None, None, None, None, None, None, None]
    duration = analysis['track']['duration']
    loudness = analysis['track']['loudness']
    tempo = analysis['track']['tempo']
    tempo_conf = analysis['track']['tempo_confidence']
    time_sig = analysis['track']['time_signature']
    time_sig_conf = analysis['track']['time_signature_confidence']
    key = analysis['track']['key']
    key_conf = analysis['track']['key_confidence']
    mode = analysis['track']['mode']
    mode_conf = analysis['track']['mode_confidence']
    return [duration, loudness, tempo, tempo_conf, time_sig, time_sig_conf, key, key_conf, mode, mode_conf]

In [284]:
duration_list = []
loudness_list = []
tempo_list = []
tempo_conf_list = []
time_sig_list = []
time_sig_conf_list = []
key_list = []
key_conf_list =[]
mode_list = []
mode_conf_list = []

for index, row in spotify_data.iterrows():
    string = re.sub('\(|\)', ", ", re.sub('\s+((ft\.)|&|X|x|(\+)|/)\s+', ", ", row['Artist'])).strip(', ')
    artist_list = string.split(",")
    values = [None, None, None, None, None, None, None, None, None, None]
    for artist in artist_list:
        if values == [None, None, None, None, None, None, None, None, None, None]:
            values = get_audio_analysis(artist, re.sub('\(.*\)', "", row['Title']))
    duration_list.append(values[0])
    loudness_list.append(values[1])
    tempo_list.append(values[2])
    tempo_conf_list.append(values[3])
    time_sig_list.append(values[4])
    time_sig_conf_list.append(values[5])
    key_list.append(values[6])
    key_conf_list.append(values[7])
    mode_list.append(values[8])
    mode_conf_list.append(values[9])

spotify_data.insert(6, "Duration", duration_list, False)
spotify_data.insert(7, "Loudness", loudness_list, False)
spotify_data.insert(8, "Tempo", tempo_list, False)
spotify_data.insert(9, "Tempo_Confidence", tempo_conf_list, False)
spotify_data.insert(10, "Meter", time_sig_list, False)
spotify_data.insert(11, "Meter_Confidence", time_sig_conf_list, False)
spotify_data.insert(12, "Key", key_list, False)
spotify_data.insert(13, "Key_Confidence", key_conf_list, False)
spotify_data.insert(14, "Mode", mode_list, False)
spotify_data.insert(15, "Mode_Confidence", mode_conf_list, False)
spotify_data.head(10)

HTTP Error for GET to https://api.spotify.com/v1/audio-analysis/4LaGu95Ui2s4vprSQYWUAZ with Params: {} returned 404 due to analysis not found
HTTP Error for GET to https://api.spotify.com/v1/audio-analysis/6yuvC80FcnVJNvC0DbXN9e with Params: {} returned 404 due to analysis not found
HTTP Error for GET to https://api.spotify.com/v1/audio-analysis/3uh7YcFzAWHGg7spVzPfqP with Params: {} returned 404 due to analysis not found
HTTP Error for GET to https://api.spotify.com/v1/audio-analysis/5gfPJ45gpn3ThswDyeW0Qc with Params: {} returned 404 due to analysis not found
HTTP Error for GET to https://api.spotify.com/v1/audio-analysis/0BXTqB4It8UM09lCaIY3Jk with Params: {} returned 404 due to analysis not found
HTTP Error for GET to https://api.spotify.com/v1/audio-analysis/5OGkKx8jP0A5KSULEc6XYZ with Params: {} returned 404 due to analysis not found
HTTP Error for GET to https://api.spotify.com/v1/audio-analysis/6MFQeWtk7kxWGydnJB2y36 with Params: {} returned 404 due to analysis not found


,Top_Rank,Title,Artist,Weeks_in_hot-100,First_Week,Last_Week,Duration,Loudness,Tempo,Tempo_Confidence,Meter,Meter_Confidence,Key,Key_Confidence,Mode,Mode_Confidence
0,22,#1,Nelly,20,2001-10-20,2002-03-02,223.08000,-6.358,116.935,0.841,4.0,1.000,1.0,0.459,1.0,0.517
1,15,#Beautiful,Mariah Carey ft. Miguel,16,2013-05-25,2013-09-07,199.94667,-5.333,107.030,0.001,4.0,1.000,4.0,0.228,1.0,0.334
2,16,#SELFIE,The Chainsmokers,11,2014-03-15,2014-05-24,183.74998,-3.262,127.956,0.835,4.0,0.923,0.0,0.508,1.0,0.467
3,17,#thatPOWER,will.i.am ft. Justin Bieber,16,2013-04-06,2013-07-20,279.50668,-6.096,127.999,0.796,4.0,1.000,6.0,0.465,0.0,0.504
4,71,$ave Dat Money,Lil Dicky ft. Fetty Wap & Rich Homie Quan,19,2015-10-10,2016-03-19,290.83570,-5.361,98.013,0.399,4.0,1.000,2.0,0.643,1.0,0.347
5,4,'03 Bonnie & Clyde,Jay-Z ft. Beyonce Knowles,23,2002-10-26,2003-03-29,205.56000,-5.148,89.640,0.723,4.0,1.000,9.0,0.672,0.0,0.564
6,58,'Til Summer Comes Around,Keith Urban,16,2010-01-30,2010-05-15,331.46667,-7.608,127.907,0.210,4.0,1.000,9.0,0.494,0.0,0.505
7,18,'Til You Can't,Cody Johnson,30,2021-10-23,2022-05-14,224.21333,-4.865,160.087,0.387,4.0,0.996,1.0,0.494,1.0,0.689
8,39,'Tis The Damn Season,Taylor Swift,2,2020-12-26,2021-01-02,229.84000,-8.193,145.916,0.362,4.0,0.720,5.0,0.693,1.0,0.585
9,7,(Hot S**t) Country Grammar,Nelly,34,2000-04-29,2000-12-16,291.78195,-6.490,101.875,0.966,4.0,1.000,7.0,0.597,1.0,0.510


In [287]:
null_data = spotify_data[spotify_data.isnull().any(axis=1)]
print(len(null_data.index))
null_data.head(10)
# should be 2
spotify_data.to_csv("csv/spotify_data_2000_01_01.csv")
print(len(spotify_data.index))

269
8486


Phase 2: Data Management and Representation

Phase 3: Exploratory Data Analysis

Phase 4: Hypothesis Testing

Phase 5: Communication of Insights Attained

Understanding: the reader of the tutorial should walk away with some new understanding of the topic at hand. If it’s not possible for a reader to state ‘what they learned’ from reading your tutorial, then why do the analysis?